In [1]:
import numpy as np
import pandas as pd

In [4]:
df = pd.read_csv("naver_review_0810.csv")

In [5]:
!pip3 install pandas nltk gensim pyLdavis

  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
    Preparing wheel metadata: started
    Preparing wheel metadata: finished with status 'done'
  Created wheel for pyLdavis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135618 sha256=3ef62c8329a8a667156849a43081155a1487c49443b7b726993dec7d99aca663
  Sto

You should consider upgrading via the 'c:\users\tkdal\anaconda3\python.exe -m pip install --upgrade pip' command.


In [7]:
import re # We clean text using regex
import csv # To read the csv
from collections import defaultdict # For accumlating values
from nltk.corpus import stopwords # To remove stopwords
from gensim import corpora # To create corpus and dictionary for the LDA model
from gensim.models import LdaModel # To use the LDA model
import pyLDAvis.gensim # To visualise LDA model effectively
import pandas as pd


C:\Users\tkdal\Anaconda3\lib\site-packages\gensim\similarities\__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [8]:
df.head()

,index,name,cate_1,cate_2,cate_3,dong,lon,lat,cate_mix,naver_keyword,naver_map_url,naver_store_type,naver_star_point,naver_star_point_qty,naver_blog_review_txt,naver_blog_review_qty
0,2140,노랑통닭,닭/오리요리,후라이드/양념치킨,치킨 전문점,흑석동,37.507850,126.960672,닭 오리요리후라이드 양념치킨치킨 전문점,흑석동%20노랑통닭,https://m.place.naver.com/restaurant/38419604,"치킨,닭강정",4.45,121.0,서울 흑석동 중앙대 치맥 맛집. 노랑통닭 순살 3종 세트 맛있어요. 2020년 9월...,25
1,2614,맥주창고,유흥주점,호프/맥주,기타 주점업,흑석동,37.508109,126.961404,유흥주점호프 맥주기타 주점업,흑석동%20맥주창고,https://m.place.naver.com/restaurant/31040500,"맥주,호프",4.50,17.0,흑석동 맥주창고에서 간단하게 맥주한잔!!! 퇴근하고 친구가 간단하게 한잔하자고해서 ...,5
2,3450,Mr.꼼장어Miss쭈꾸미,일식/수산물,곰장어전문,한식 음식점업,흑석동,37.508488,126.962541,일식 수산물곰장어전문한식 음식점업,흑석동%20Mr.꼼장어Miss쭈꾸미,https://m.place.naver.com/restaurant/12880938,"해물,생선요리",4.64,18.0,흑석동 중앙대 맛집 미스터꼼장어 미스쭈꾸미 얼마 전 처음으로 ! 꼼장어를 먹어보게 ...,16
3,6254,가춘,제과제빵떡케익,제과점,제과점업,상도1동,37.496929,126.953495,제과제빵떡케익제과점제과점업,상도1동%20가춘,https://m.place.naver.com/restaurant/1123429741,베이커리,4.54,369.0,상도곱창 바로 옆에 있는 #빵집 #가춘 딱봐도 #맛집 느낌이길래 상도사는 친구에게 ...,66
4,7129,BHC치킨,닭/오리요리,후라이드/양념치킨,치킨 전문점,상도1동,37.494928,126.957436,닭 오리요리후라이드 양념치킨치킨 전문점,상도1동%20BHC치킨,https://m.place.naver.com/restaurant/37413677,"치킨,닭강정",4.47,195.0,bhc 포테킹 후라이드 신메뉴 가격 요기요앱에서 할인 중 ! @bhc 일요일은 거의...,9


In [9]:
reviews = df['naver_blog_review_txt']

In [11]:
reviews = [re.sub(r'[^\w\s]','',str(item)) for item in reviews]

In [13]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

stopwords = "아 휴 아이구 아이쿠 아이고 어 나 우리 저희 따라 의해 을 를 에 의 가 으로 로 에게 뿐이다 의거하여 근거하여 입각하여 기준으로 예하면 예를 들면 예를 들자면 저 소인 소생 저희 지말고 하지마 하지마라 다른 물론 또한 그리고 비길수 없다 해서는 안된다 뿐만 아니라 만이 아니다 만은 아니다 막론하고 관계없이 그치지 않다 그러나 그런데 하지만 든간에 논하지 않다 따지지 않다 설사 비록 더라도 아니면 만 못하다 하는 편이 낫다 불문하고 향하여 향해서 향하다 쪽으로 틈타 이용하여 타다 오르다 제외하고 이 외에 이 밖에 하여야 비로소 한다면 몰라도 외에도 이곳 여기 부터 기점으로 따라서 할 생각이다 하려고하다 이리하여 그리하여 그렇게 함으로써 하지만 일때 할때 앞에서 중에서 보는데서 으로써 로써 까지 해야한다 일것이다 반드시 할줄알다 할수있다 할수있어 임에 틀림없다 한다면 등 등등 제 겨우 단지 다만 할뿐 딩동 댕그 대해서 대하여 대하면 훨씬 얼마나 얼마만큼 얼마큼 남짓 여 얼마간 약간 다소 좀 조금 다수 몇 얼마 지만 하물며 또한 그러나 그렇지만 하지만 이외에도 대해 말하자면 뿐이다 다음에 반대로 반대로 말하자면 이와 반대로 바꾸어서 말하면 바꾸어서 한다면 만약 그렇지않으면 까악 툭 딱 삐걱거리다 보드득 비걱거리다 꽈당 응당 해야한다 에 가서 각 각각 여러분 각종 각자 제각기 하도록하다 와 과 그러므로 그래서 고로 한 까닭에 하기 때문에 거니와 이지만 대하여 관하여 관한 과연 실로 아니나다를가 생각한대로 진짜로 한적이있다 하곤하였다 하 하하 허허 아하 거바 와 오 왜 어째서 무엇때문에 어찌 하겠는가 무슨 어디 어느곳 더군다나 하물며 더욱이는 어느때 언제 야 이봐 어이 여보시오 흐흐 흥 휴 헉헉 헐떡헐떡 영차 여차 어기여차 끙끙 아야 앗 아야 콸콸 졸졸 좍좍 뚝뚝 주룩주룩 솨 우르르 그래도 또 그리고 바꾸어말하면 바꾸어말하자면 혹은 혹시 답다 및 그에 따르는 때가 되어 즉 지든지 설령 가령 하더라도 할지라도 일지라도 지든지 몇 거의 하마터면 인젠 이젠 된바에야 된이상 만큼 어찌됏든 그위에 게다가 점에서 보아 비추어 보아 고려하면 하게될것이다 일것이다 비교적 좀 보다더 비하면 시키다 하게하다 할만하다 의해서 연이서 이어서 잇따라 뒤따라 뒤이어 결국 의지하여 기대여 통하여 자마자 더욱더 불구하고 얼마든지 마음대로 주저하지 않고 곧 즉시 바로 당장 하자마자 밖에 안된다 하면된다 그래 그렇지 요컨대 다시 말하자면 바꿔 말하면 즉 구체적으로 말하자면 시작하여 시초에 이상 허 헉 허걱 바와같이 해도좋다 해도된다 게다가 더구나 하물며 와르르 팍 퍽 펄렁 동안 이래 하고있었다 이었다 에서 로부터 까지 예하면 했어요 해요 함께 같이 더불어 마저 마저도 양자 모두 습니다 가까스로 하려고하다 즈음하여 다른 다른 방면으로 해봐요 습니까 했어요 말할것도 없고 무릎쓰고 개의치않고 하는것만 못하다 하는것이 낫다 매 매번 들 모 어느것 어느 로써 갖고말하자면 어디 어느쪽 어느것 어느해 어느 년도 라 해도 언젠가 어떤것 어느것 저기 저쪽 저것 그때 그럼 그러면 요만한걸 그래 그때 저것만큼 그저 이르기까지 할 줄 안다 할 힘이 있다 너 너희 당신 어찌 설마 차라리 할지언정 할지라도 할망정 할지언정 구토하다 게우다 토하다 메쓰겁다 옆사람 퉤 쳇 의거하여 근거하여 의해 따라 힘입어 그 다음 버금 두번째로 기타 첫번째로 나머지는 그중에서 견지에서 형식으로 쓰여 입장에서 위해서 단지 의해되다 하도록시키다 뿐만아니라 반대로 전후 전자 앞의것 잠시 잠깐 하면서 그렇지만 다음에 그러한즉 그런즉 남들 아무거나 어찌하든지 같다 비슷하다 예컨대 이럴정도로 어떻게 만약 만일 위에서 서술한바와같이 인 듯하다 하지 않는다면 만약에 무엇 무슨 어느 어떤 아래윗 조차 한데 그럼에도 불구하고 여전히 심지어 까지도 조차도 하지 않도록 않기 위하여 때 시각 무렵 시간 동안 어때 어떠한 하여금 네 예 우선 누구 누가 알겠는가 아무도 줄은모른다 줄은 몰랏다 하는 김에 겸사겸사 하는바 그런 까닭에 한 이유는 그러니 그러니까 때문에 그 너희 그들 너희들 타인 것 것들 너 위하여 공동으로 동시에 하기 위하여 어찌하여 무엇때문에 붕붕 윙윙 나 우리 엉엉 휘익 윙윙 오호 아하 어쨋든 만 못하다 하기보다는 차라리 하는 편이 낫다 흐흐 놀라다 상대적으로 말하자면 마치 아니라면 쉿 그렇지 않으면 그렇지 않다면 안 그러면 아니었다면 하든지 아니면 이라면 좋아 알았어 하는것도 그만이다 어쩔수 없다 하나 일 일반적으로 일단 한켠으로는 오자마자 이렇게되면 이와같다면 전부 한마디 한항목 근거로 하기에 아울러 하지 않도록 않기 위해서 이르기까지 이 되다 로 인하여 까닭으로 이유만으로 이로 인하여 그래서 이 때문에 그러므로 그런 까닭에 알 수 있다 결론을 낼 수 있다 으로 인하여 있다 어떤것 관계가 있다 관련이 있다 연관되다 어떤것들 에 대해 이리하여 그리하여 여부 하기보다는 하느니 하면 할수록 운운 이러이러하다 하구나 하도다 다시말하면 다음으로 에 있다 에 달려 있다 우리 우리들 오히려 하기는한데 어떻게 어떻해 어찌됏어 어때 어째서 본대로 자 이 이쪽 여기 이것 이번 이렇게말하자면 이런 이러한 이와 같은 요만큼 요만한 것 얼마 안 되는 것 이만큼 이 정도의 이렇게 많은 것 이와 같다 이때 이렇구나 것과 같이 끼익 삐걱 따위 와 같은 사람들 부류의 사람들 왜냐하면 중의하나 오직 오로지 에 한하다 하기만 하면 도착하다 까지 미치다 도달하다 정도에 이르다 할 지경이다 결과에 이르다 관해서는 여러분 하고 있다 한 후 혼자 자기 자기집 자신 우에 종합한것과같이 총적으로 보면 총적으로 말하면 총적으로 대로 하다 으로서 참 그만이다 할 따름이다 쿵 탕탕 쾅쾅 둥둥 봐 봐라 아이야 아니 와아 응 아이 참나 년 월 일 령 영 일 이 삼 사 오 육 륙 칠 팔 구 이천육 이천칠 이천팔 이천구 하나 둘 셋 넷 다섯 여섯 일곱 여덟 아홉 령 영 이 있 하 것 들 그 되 수 이 보 않 없 나 사람 주 아니 등 같 우리 때 년 가 한 지 대하 오 말 일 그렇 위하 때문 그것 두 말하 알 그러나 받 못하 일 그런 또 문제 더 사회 많 그리고 좋 크 따르 중 나오 가지 씨 시키 만들 지금 생각하 그러 속 하나 집 살 모르 적 월 데 자신 안 어떤 내 내 경우 명 생각 시간 그녀 다시 이런 앞 보이 번 나 다른 어떻 여자 개 전 들 사실 이렇 점 싶 말 정도 좀 원 잘 통하 놓 "

#word_tokens = word_tokenize(review)
stopwords=stopwords.split(' ')


In [ ]:

result = [] 
for w in word_tokens: 
    if w not in stop_words: 
        result.append(w) 

print(word_tokens,'\n') 
print(result)

In [14]:
texts = [[word for word in document.lower().split() if word not in stopwords] for document in reviews]


In [15]:
frequency = defaultdict(int)

for text in texts:
    for token in text:
        frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]

In [16]:
dictionary = corpora.Dictionary(texts)
print(dictionary)

Dictionary(8008 unique tokens: ['2020년', '3종', 'ㅋㅋ', 'ㅋㅋㅋ', 'ㅋㅋㅋㅋ']...)


In [17]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [18]:
# This is an assumption.
NUM_TOPICS = 9 

#This might take some time.
ldamodel = LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15)

In [20]:
topics = ldamodel.show_topics()

for topic in topics:
    print(topic)

(0, '0.006*"맛집" + 0.005*"숭실대" + 0.005*"있는" + 0.004*"너무" + 0.004*"오늘은" + 0.004*"상도동" + 0.004*"떡볶이" + 0.004*"뚜레쥬르" + 0.004*"image" + 0.004*"안녕하세요"')
(1, '0.006*"맛집" + 0.006*"오늘은" + 0.005*"있는" + 0.004*"상도동" + 0.004*"흑석동" + 0.004*"너무" + 0.004*"많이" + 0.004*"상도역" + 0.003*"입니다" + 0.003*"정말"')
(2, '0.007*"카페" + 0.006*"너무" + 0.006*"있는" + 0.005*"케이크" + 0.005*"오늘은" + 0.004*"중앙대" + 0.004*"많이" + 0.004*"맛집" + 0.004*"상도동" + 0.004*"정말"')
(3, '0.009*"중앙대" + 0.009*"있는" + 0.008*"맛집" + 0.007*"오늘은" + 0.004*"오랜만에" + 0.004*"흑석동" + 0.004*"위치한" + 0.004*"많이" + 0.004*"너무" + 0.004*"먹고"')
(4, '0.007*"있는" + 0.006*"오늘은" + 0.006*"숭실대" + 0.004*"너무" + 0.004*"맛집" + 0.004*"근처에" + 0.004*"저는" + 0.004*"진짜" + 0.004*"푸라닭" + 0.003*"서울"')
(5, '0.007*"오늘은" + 0.007*"있는" + 0.007*"맛집" + 0.005*"카페" + 0.005*"너무" + 0.004*"많이" + 0.004*"먹고" + 0.004*"숭실대" + 0.004*"제가" + 0.003*"흑석동"')
(6, '0.006*"오늘" + 0.006*"오늘은" + 0.004*"먹고" + 0.004*"있는" + 0.004*"맘스터치" + 0.004*"집에서" + 0.004*"먹는" + 0.003*"메뉴" + 0.003*"가격은" + 0.003*"제가"')
(7, '0.009*"있는" 

In [22]:
word_dict = {};

for i in range(NUM_TOPICS):
    words = ldamodel.show_topic(i, topn = 20)
    word_dict['Topic # ' + '{:02d}'.format(i+1)] = [i[0] for i in words]

pd.DataFrame(word_dict)

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09
0,맛집,맛집,카페,중앙대,있는,오늘은,오늘,있는,있는
1,숭실대,오늘은,너무,있는,오늘은,있는,오늘은,중앙대,흑석동
2,있는,있는,있는,맛집,숭실대,맛집,먹고,맛집,오늘은
3,너무,상도동,케이크,오늘은,너무,카페,있는,흑석동,중앙대
4,오늘은,흑석동,오늘은,오랜만에,맛집,너무,맘스터치,오늘은,너무
5,상도동,너무,중앙대,흑석동,근처에,많이,집에서,흑석역,맛집
6,떡볶이,많이,많이,위치한,저는,먹고,먹는,정말,안녕하세요
7,뚜레쥬르,상도역,맛집,많이,진짜,숭실대,메뉴,근처에,먹고
8,image,입니다,상도동,너무,푸라닭,제가,가격은,좋은,카페
9,안녕하세요,정말,정말,먹고,서울,흑석동,제가,학교,많이
